Cleansing the dataset of the loaded images from the garbage files using the mediapipe for detecting faces.

For running the code we should have two folders in the working directory: <br>
input_folder = "wiki_crop" - with the images to be processed, unpacked from wiki_crop.tar with the size 774 MB <br>
output_folder = "wiki_crop_cleaned" - empty folder for processed images <br>

1. Importing necessary modules:

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np

2. Defining functions for loading images and detecting faces on them

In [ ]:
def load_images_from_folder(folder):
    """Загрузка всех графических файлов из папки и её подпапок."""
    image_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_paths.append(os.path.join(root, file))
    return image_paths

def detect_faces_and_filter_images(image_paths, output_folder, min_face_size=100):
    """Обнаружение лиц и сохранение изображений, где лица достаточно большие."""
    mp_face_detection = mp.solutions.face_detection.FaceDetection()

    # Убедимся, что выходная папка существует
    os.makedirs(output_folder, exist_ok=True)

    for image_path in image_paths:
        # Открытие изображения через OpenCV
        img = cv2.imread(image_path)
        if img is None:  # Если изображение не загружено, пропускаем его
            print(f"Не удалось загрузить изображение: {image_path}")
            continue
        
        # Преобразование из BGR в RGB (mediapipe ожидает RGB)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape

        # Обнаружение лиц
        results = mp_face_detection.process(img_rgb)

        if results.detections:
            for detection in results.detections:
                # Извлечение размеров обнаруженного лица
                bboxC = detection.location_data.relative_bounding_box
                face_width = int(bboxC.width * width)
                face_height = int(bboxC.height * height)

                # Проверка на минимальный размер лица
                if face_width >= min_face_size and face_height >= min_face_size:
                    # Сохранение изображения в выходную папку
                    output_path = os.path.join(output_folder, os.path.basename(image_path))
                    # resize image to 224 x 224:
                    I = cv2.resize(img, (224, 224)) # size for EfficientNet
                    cv2.imwrite(output_path, I)
                    print(f"Сохранено изображение: {output_path}")
                    break  # Сохраняем один раз, если есть подходящее лицо

3. Loading images one by one, detecting images and saving them to another folder if the face is large enough

In [ ]:
# Папка с изображениями и выходная папка
input_folder = "wiki_crop"
output_folder = "wiki_crop_cleaned"

# Загрузка изображений и их фильтрация
min_face_size = 100
image_paths = load_images_from_folder(input_folder)
detect_faces_and_filter_images(image_paths, output_folder, min_face_size=min_face_size)